### Introduction to Tensorflow

In [1]:
import tensorflow as tf

# constant nodes -> they don't have an input - example : a bias
node1 = tf.constant(3.0, dtype=tf.float32, name="node1")
node2 = tf.constant(4.0, name="blablahhh") # also tf.float32 implicitly
node3 = tf.constant(5.0)

# this will print the nodes' characteristics not the constants
print(node1, node2, node3)

Tensor("node1:0", shape=(), dtype=float32) Tensor("blablahhh:0", shape=(), dtype=float32) Tensor("Const:0", shape=(), dtype=float32)


In [2]:
# let's start a session and evaluate the computational graph :
sess = tf.Session()
print(sess.run([node1, node2, node3])) # will print the node's outputs

[3.0, 4.0, 5.0]


We can build more complicated computations by combining Tensor nodes with operations (Operations are also nodes). 
For example, we can add our two constant nodes and produce a new graph as follows:

In [3]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

In [4]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

The preceding three lines are a bit like a function or a lambda in which we define two input parameters (a and b) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument to the run method to feed concrete values to the placeholders:

In [5]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


We can make the computational graph more complex by adding another operation. For example,

In [6]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [7]:
W = tf.Variable([.3], dtype=tf.float32) # Variables can be initialized - not to be confused with constants though -
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [19]:
init = tf.global_variables_initializer() # this was not needed before, since we did not have constants but variables -
sess.run(init) # initializes the variables

init is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call sess.run, the variables are uninitialized.

Since x is a placeholder, we can evaluate linear_model for **several values** of x simultaneously as follows:

In [20]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


We've created a model, but we don't know how good it is yet. To evaluate the model on training data, we need a y placeholder to provide the desired values, and we need to write a loss function.

A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum:

In [25]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y) # works with tensors of any shape
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

# if y matches exactly the results of linear_model calculated above - then loss function is at 0
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, 0.3, 0.6, 0.90000004]}))


5.69997e-11
23.66


We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. A variable is initialized to the value provided to tf.Variable but can be changed using operations like tf.assign. For example, W=-1 and b=1 are the optimal parameters for our model. We can change W and b accordingly:

In [26]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb]) # initializes the fixW and fixb variables
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})) # 0 - boom !

0.0


We guessed the "perfect" values of W and b, but the whole point of machine learning is to find the correct model parameters automatically. We will show how to accomplish this in the next section.

### tf.train API
A complete discussion of machine learning is out of the scope of this tutorial. However, TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is gradient descent. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. In general, computing symbolic derivatives manually is tedious and error-prone. Consequently, TensorFlow can automatically produce derivatives given only a description of the model using the function tf.gradients. For simplicity, optimizers typically do this for you. For example,

In [28]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b])) # print Weights and bias

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


#### sess.run takes 2 args, a function (e.g. loss, adder, train etc) and the arguments to this function, tensors.

Now we have done actual machine learning! Although this simple linear regression model does not require much TensorFlow core code, more complicated models and methods to feed data into your models necessitate more code. Thus, TensorFlow provides higher level abstractions for common patterns, structures, and functionality. We will learn how to use some of these abstractions in the next section.

In [41]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], name="W") # dtype defaults to tf.float32 - not required here or below
b = tf.Variable([-.3], dtype=tf.float32)

# Model input and output
x = tf.placeholder(tf.float32) # here the dtype definition is mandatory
linear_model = W*x + b # linear_model is nothing else but a tensor

y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # Classic SSE loss - Sum of Squared Errors

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01) # optimizer algo based on gradient descent with alpha 0.01
train = optimizer.minimize(loss) # train is a node that will minimize the loss based on above algo, gradient descent

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})
    
    if i%100 == 0:
        print("at step ", i)

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

at step  0
at step  100
at step  200
at step  300
at step  400
at step  500
at step  600
at step  700
at step  800
at step  900
W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


### tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning :

* running training loops
* running evaluation loops
* managing data sets
* tf.estimator defines many common models.

### Basic usage

Notice how much simpler the linear regression program becomes with tf.estimator:

In [42]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_tf_random_seed': None, '_is_chief': True, '_model_dir': '/var/folders/9c/whwt0lds4pzcwpmd_82qyy0h0000gn/T/tmpzid2zsai', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10731e898>, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_task_id': 0, '_master': '', '_num_worker_replicas': 1, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_service': None}


INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_tf_random_seed': None, '_is_chief': True, '_model_dir': '/var/folders/9c/whwt0lds4pzcwpmd_82qyy0h0000gn/T/tmpzid2zsai', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10731e898>, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_task_id': 0, '_master': '', '_num_worker_replicas': 1, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_service': None}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2018-01-29-05:57:52


INFO:tensorflow:Finished evaluation at 2018-01-29-05:57:52


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.0025789, global_step = 1000, loss = 0.0103156


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.0025789, global_step = 1000, loss = 0.0103156


train metrics: {'global_step': 1000, 'loss': 9.5268388e-07, 'average_loss': 2.3817097e-07}
eval metrics: {'global_step': 1000, 'loss': 0.010315614, 'average_loss': 0.0025789035}
